In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

#######################################################################
df1 = pd.read_csv('result/df1.csv')
df2 = pd.read_csv('result/df2.csv')
df3 = pd.read_csv('result/df3.csv')
# df4 = pd.read_csv('/opt/ml/result/hardvoting_result/fold3_9718.csv')
# df5 = pd.read_csv('/opt/ml/result/hardvoting_result/fold4_9723.csv')

# df_list = [df1, df2, df3, df4, df5]
df_list = [df1, df2, df3]
#######################################################################

In [36]:
# mask map으로 나오는 인퍼런스 결과를 RLE로 인코딩 합니다. (mask map -> RLE)
def encode_mask_to_rle(mask):
    '''
    mask: numpy array binary mask 
    1 - mask 
    0 - background
    Returns encoded run length 
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# RLE로 인코딩된 결과를 mask map으로 복원합니다. (RLE -> mask map)
def decode_rle_to_mask(rle, height, width):
    s = str(rle).split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

In [37]:
threshold = len(df_list)//2

# 29 * 288
# df length를 29개씩 끊어서 처리
for i in tqdm(range(0, len(df1), 29)):
    # class number
    for j in range(29):
        result = np.zeros((2048, 2048), dtype=int)

        # ensemble candidates
        for idx, df in enumerate(df_list):
            rle = df.iloc[i+j].rle
            try:
                mask = decode_rle_to_mask(rle, 2048, 2048) # 01 matrix
                result += mask
            except:
                continue
            
        # calculate with threshold
        result = np.where(result > threshold, 1, 0)
        df_list[0].rle[i+j] = encode_mask_to_rle(result)
        
df_list[0].to_csv("hardvote_result.csv", index=False)

  0%|          | 0/288 [00:00<?, ?it/s]/tmp/ipykernel_603679/2587036645.py:21: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_list[0].rle[i+j] = encode_mask_to_rle(result)
100%|██████████| 288/288 [17:13<00:00,  3.59s/it]
